In [ ]:
!nvidia-smi

In [6]:
import os
import torch
from tqdm.notebook import tqdm
import sys
sys.path.append('RULEARN_IDEA')

# transformers Model save path & gpu setup
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1" # set your gpus
os.environ["TRANSFORMERS_CACHE"] = '../hg_model_cache'
if torch.cuda.is_available():
    print("available num GPU", torch.cuda.device_count())
    
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print("  - Total G mem:", torch.cuda.get_device_properties(i).total_memory / (1024 ** 3), "GB")
        print("  - #Cores:", torch.cuda.get_device_properties(i).multi_processor_count)
    device = 'cuda'
else:
    print(f'''No GPUs found in current environment''')
    print("No GPU.")
    device = 'cpu'

from typing import *
import random
from collections import deque
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Heiti TC'] 
plt.rcParams['axes.unicode_minus'] = False 
from abc import ABC, abstractmethod
import time
import ast
import re
import copy
import pandas as pd
import datetime
import logging
import shutil
import concurrent.futures
import traceback

# CHIBI framework components
import world_basic_blocks as blocks
import space_manager as sm
import utils
import CHIBI
import plan_system
import fixed_interactive_pipeline_objects as fixed_blocks
import threading

from Judger import Judger

# Story profile
from all_puzzle_settings import *

# OPEN AI APIs
# openai api keys
import openai
from openai import AsyncOpenAI

No GPUs found in current environment
No GPU.


In [3]:
def generate_puzzle_spaces(puzzle_setting_file:Dict[str, Any],
                           puzzle_level:int = 1,
                           puzzle_index:int = 1,
                           Model_name:str = 'gpt-3.5-turbo-0125')->List[blocks.Space_Base]:
    puzzle_level_key = 'Level'+str(puzzle_level)
    puzzle_index_key = 'puzzle'+str(puzzle_index)
    puzzle_setting = puzzle_setting_file[puzzle_level_key][puzzle_index_key]
    spaces =  sm.Space_helper.generate_all_room_with_database(puzzle_setting['Map'],
                                                              puzzle_setting['Space_items'],
                                                              puzzle_setting['Space_item_containers'],
                                                              puzzle_setting['Edges'],
                                                              Model_name = Model_name)
    Space_Manager_System_Global = sm.Space_Manager_System(spaces)
    CHIBI_profile = puzzle_setting['Agent']
    return Space_Manager_System_Global, CHIBI_profile, puzzle_setting

def init_puzzle(puzzle_setting_file_name:str,
                puzzle_level:int,
                puzzle_index:int,
                Do_abduction:bool = False,
                Model_name:str = 'gpt-3.5-turbo-0125',
                human_test_bool:bool = False,
                memory_buffer_size:int = 0,
                CHIBI_name:Optional[str] = None,
                Batch_generator:Optional[utils.Prompt_batch_generator] = None,
                forced_abduction:bool = False):
    
    puzzle_dict = {'Reactor_puzzles':Reactor_puzzles,
                   'Art_gallery_puzzles':Art_gallery_puzzles,
                   'Function_operator_puzzles':Function_operator_puzzles}
    
    puzzle_setting_file = puzzle_dict[puzzle_setting_file_name]
    Space_Manager_System_Global, CHIBI_setting, puzzle_setting = generate_puzzle_spaces(puzzle_setting_file, 
                                                                        puzzle_level, 
                                                                        puzzle_index, 
                                                                        Model_name = Model_name)
    CHIBI_profile = CHIBI.CHIBI_helper.create_profile_with_legacy_file(CHIBI_setting, CHIBI_name = CHIBI_name)
    if puzzle_setting_file_name == 'Reactor_puzzles':
        CHIBI_profile.Current_situation += f''' In this puzzle, you need to explore the patterns of reaction by conducting continuous experiments(The law is simple and can be described in one sentense). Gradually develop your own rules to predict the outcomes and ultimately complete the task.'''
        if puzzle_level == 1: # add rule for level 1 puzzle
            if 1<=puzzle_index<=5:
                CHIBI_profile.Current_situation += f'''The reaction is straight forward, simply combine two materials in the order you put them into the <Reactor>. Eg if you put A,B into the reactor it produce AB as output. Please use this rule to guide your action.'''
            elif 6<=puzzle_index<=10:
                CHIBI_profile.Current_situation += f'''The reaction is straight forward, simply combine two materials in the reverse order you put them into the <Reactor>. Eg if you put A,B into the reactor it have BA are output. If you put AC, BD into the reactor it produces BDAC. Please use this rule to guide your action.'''
            elif 11<=puzzle_index<=15:
                CHIBI_profile.Current_situation += f'''The reaction insert the shorter material into the middle of the longer material. Eg, If you put AB and C, it produces ACB. Please use this rule to guide your action.'''
            elif 16<=puzzle_index<=20:
                CHIBI_profile.Current_situation += f'''In this reaction, when two materials of different lengths are combined, the longer material's initial character is retained while the remainder of it is substituted with the shorter material. The replaced segment of the longer material is then kept alongside the newly formed product. However, there are two exceptions to this rule: 1. If the two materials are of the same length, they are simply concatenated. 2. If each material consists of only one letter, they are also simply concatenated. For example, AB+C = AC+B, AC+DD = ACDD. Please use this rule to guide your action.'''
        elif puzzle_level == 2:
            if 1<=puzzle_index<=5:
                CHIBI_profile.Current_situation += f'''You know from an incomplete list of reaction equations that: XY+Z = XYZ. X, Y, and Z are placeholders representing different reactants or products, not specific substances.'''
            elif 6<=puzzle_index<=10:
                CHIBI_profile.Current_situation += f'''You know from an incomplete list of reaction equations that: XY+Z = ZXY. X, Y, and Z are placeholders representing different reactants or products, not specific substances.'''
            elif 11<=puzzle_index<=15:
                CHIBI_profile.Current_situation += f'''You know from an incomplete list of reaction equations that: XY+Z = XZY. X, Y, and Z are placeholders representing different reactants or products, not specific substances.'''
            elif 16<=puzzle_index<=20:
                CHIBI_profile.Current_situation += f'''You know from an incomplete list of reaction equations that: XY+Z = XZ + Y. X, Y, and Z are placeholders representing different reactants or products, not specific substances.'''
            
    elif puzzle_setting_file_name == 'Art_gallery_puzzles':
        CHIBI_profile.Current_situation = f'''In this puzzle, set in an art gallery, {CHIBI_name} must uncover the password for the <Code Secured Door> by discovering the relationships between the password and the paintings. And finally input the password into the <Code Secured door>.'''
        if puzzle_level == 1:
            CHIBI_profile.Current_situation += " Currently, you know that the 3-digit password for the <Code Secured Door> is determined as follows: the first digit corresponds to the number of oil paintings in a specific color, the second digit to the number of acrylic paintings in that color, and the third digit to the number of watercolor paintings in the same color."
        elif puzzle_level == 2:
            CHIBI_profile.Current_situation += ''' Currently, you see from a note on the ground that says: "Focus on blue it hides the truth."'''
        CHIBI_profile.Current_situation += f'''You can test your assumption by entering the password into the door. However, be aware that if you exceed the attempt limit, the password and hint will change.'''
    elif puzzle_setting_file_name == 'Function_operator_puzzles':
        CHIBI_profile.Current_situation += f'''You can test your assumption by entering values into the door. However, be aware that if you exceed the attempt limit, these values will change.'''
    if not human_test_bool:
        CHIBI_agent = CHIBI.CHIBI_main_character(CHIBI_profile,Space_Manager_System_Global,
                                                 Init_position = CHIBI_setting['Init_position'],
                                                 Model_name = Model_name, Do_abduction = Do_abduction, 
                                                 Special_label = puzzle_setting_file_name,
                                                 forced_abduction = forced_abduction)
    else:
        CHIBI_agent = CHIBI.CHIBI_Human(CHIBI_profile, Space_Manager_System_Global,
                                        Init_position = CHIBI_setting['Init_position'],
                                        Model_name = Model_name, Do_abduction = Do_abduction) # human agent do not need forced abduction
    CHIBI_agent.Memory_stream.Buffer_size = memory_buffer_size
    new_state = (f'{CHIBI_agent.Profile.Current_situation}',)
    for i in new_state:
        CHIBI_agent.Plan_system.add_state(i)
    return Space_Manager_System_Global,CHIBI_agent, puzzle_setting

In [4]:
Model_name = 'gpt-4o-2024-05-13'
max_length = 512
token = 'your hugging face token here'                                           # provide your huggingface token here
openai.api_key = "your openai api token"                                    # provide your openai api token here
openai.organization = "your openai org"                      # your organization here if necessary
openai_models = ['gpt-4o-2024-05-13','gpt-3.5-turbo-0125','gpt-4o-2024-08-06']   # Add your model here if you want to use open ai model

# initialize puzzle
puzzle_name = 'Reactor_puzzles' # pick one from [Reactor_puzzles, Art_gallery_puzzles, Function_operator_puzzles]
puzzle_level = 2 # input 1 or 2, 1 for oracle setting and the rule will be hiden if level is 2
puzzle_index = 12 # pick from 1 to 20
Do_abduction = True # True or False, only works in level 2
assert Do_abduction and puzzle_level != 1,f'''Do_abduction should only works when puzzle_level == 2'''
human_test_bool = False # True or False, If you want to do the puzzle yourself, please the humantest.py 
assert not human_test_bool, f'''Please run the humantest.py for trying the puzzles yourself'''
CHIBI_name = 'Kevin' # Provide a random name (No actual effect in the puzzle)
memory_buffer_size = 0 # Currently not in use don't change this
Batch_generator = None # For run a batch of puzzles in parallel
forced_abduction = None # Currently not in use don't change this
max_step_count = 15 


Space_Manager_System_Global, CHIBI_agent, puzzle_setting = init_puzzle(puzzle_name, 
                                                                       puzzle_level = puzzle_level, puzzle_index = puzzle_index,
                                                                       Do_abduction = Do_abduction, 
                                                                       Model_name = Model_name,
                                                                       human_test_bool = human_test_bool, 
                                                                       CHIBI_name = CHIBI_name,
                                                                       memory_buffer_size = memory_buffer_size,
                                                                       Batch_generator = Batch_generator,
                                                                       forced_abduction = forced_abduction,)
all_state_machine_objects = []
for single_space in Space_Manager_System_Global.Vertices_dict.values():
    objects_in_this_space = single_space.retrieve_item_in_this_space(object_type = 'All')
    for single_object in objects_in_this_space:
        if isinstance(single_object, fixed_blocks.State_machine_objects_Base):
            all_state_machine_objects.append(single_object)

if Model_name not in openai_models:
    # loading huggingface models
    from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
    tokenizer = AutoTokenizer.from_pretrained(Model_name, token = token)
    model = AutoModelForCausalLM.from_pretrained(Model_name, device_map="auto", token = token, torch_dtype= torch.bfloat16).eval()
    model = torch.compile(model)
    tokenizer.pad_token = tokenizer.eos_token 
    CHIBI_agent.Huggingface_model = model
    CHIBI_agent.Huggingface_tokenizer = tokenizer
    CHIBI_agent.Huggingface_generate_max_len = max_length


In [5]:
step_count = 0
while step_count < max_step_count:
    CHIBI_agent.Plan_system.generate_actions()
    for single_object in all_state_machine_objects:
        single_object.update()
    try:
        return_action = CHIBI_agent.Plan_system.get_action()
        if not isinstance(return_action, plan_system.Attemptation_Perceptual_Action):
            print('******************************************************')
            print(f'''************Current puzzle progress: {step_count}/{max_step_count}*************''')
            print(f'''Cur_puzzle_name: {puzzle_name}, Cur_puzzle_level: {puzzle_level}, Cur_puzzle_index:{puzzle_index},Cur_buffer_size:{memory_buffer_size}, Cur_Model_name: {Model_name}, Do_abduction:{Do_abduction}, Forced_abduction:{forced_abduction}''')
            print('******************************************************')
    except IndexError: 
        # when matching do not find the correct action index, do not follow the correct output format
        # eg: no parentheses bracket finded in the output
        # get_action_index_error_count += 1
        step_count += 1 
        continue
    except ValueError: 
        # when matching do not find the correct action index, do not follow the correct output format
        # eg: Therefore, the best choice is to **[4. Input code to the Code Secured Door and try opening it]**.
        # get_action_value_error_count += 1
        step_count += 1
        traceback.print_exc()
        continue
    except TypeError as e:
        # ("An error occurred: ", e)
        # get_action_value_error_count += 1
        step_count += 1
        traceback.print_exc()
        continue
    except utils.GenerateErrorException:
        traceback.print_exc()
        # Drop_this_puzzle = True
        break
    try:
        return_action()
        if isinstance(return_action, plan_system.Attemptation_Perceptual_Action):
            print('A perceptual action called.')
        elif isinstance(return_action, plan_system.Attemptation_Abduction_Action):
            print('A abduction action called.')
            #abduction_count += 1
            # abduction_distribution.append(str(step_count))
        else:
            step_count += 1
        CHIBI_agent.Plan_system.Previous_called_action = return_action

    except IndexError:
        # return_action_call_index_error_count += 1
        step_count += 1
        traceback.print_exc()
        continue
    except TypeError as e:
        # ("An error occurred: ", e)
        # return_action_call_index_error_count += 1
        step_count += 1
        traceback.print_exc()
        continue
    except SyntaxError as e:
        # return_action_call_index_error_count += 1
        step_count += 1
        # CHIBI_agent.Memory_stream.memory_add(e.msg)
        continue
    except utils.TaskCompletedException:
        # # mission completed
        # solution_found = True
        break
    except utils.TaskFailedException:
        break
    except utils.GenerateErrorException:
        traceback.print_exc()
        # Drop_this_puzzle = True
        break


A perceptual action called.
A perceptual action called.
******************************************************
************Current puzzle progress: 0/15*************
Cur_puzzle_name: Reactor_puzzles, Cur_puzzle_level: 2, Cur_puzzle_index:12,Cur_buffer_size:0, Cur_Model_name: gpt-4o-2024-05-13, Do_abduction:True, Forced_abduction:None
******************************************************
------------------------------------------------Prompt------------------------------------------------
You are Kevin. You need to research and generate the corresponding chemical material required in the <Task Monitor> In this puzzle, you need to explore the patterns of reaction by conducting continuous experiments(The law is simple and can be described in one sentense). Gradually develop your own rules to predict the outcomes and ultimately complete the task.You know from an incomplete list of reaction equations that: XY+Z = XZY. X, Y, and Z are placeholders representing different reactants or product

Input anything to continue!
******************************************************
************Current puzzle progress: 0/15*************
Cur_puzzle_name: Reactor_puzzles, Cur_puzzle_level: 2, Cur_puzzle_index:12,Cur_buffer_size:0, Cur_Model_name: gpt-4o-2024-05-13, Do_abduction:True, Forced_abduction:None
******************************************************
------------------------------------------------Prompt------------------------------------------------
You are Kevin. You need to research and generate the corresponding chemical material required in the <Task Monitor> In this puzzle, you need to explore the patterns of reaction by conducting continuous experiments(The law is simple and can be described in one sentense). Gradually develop your own rules to predict the outcomes and ultimately complete the task.You know from an incomplete list of reaction equations that: XY+Z = XZY. X, Y, and Z are placeholders representing different reactants or products, not specific substances.


Input anything to continue!
******************************************************
************Current puzzle progress: 1/15*************
Cur_puzzle_name: Reactor_puzzles, Cur_puzzle_level: 2, Cur_puzzle_index:12,Cur_buffer_size:0, Cur_Model_name: gpt-4o-2024-05-13, Do_abduction:True, Forced_abduction:None
******************************************************
------------------------------------------------Prompt------------------------------------------------
You are Kevin. You need to research and generate the corresponding chemical material required in the <Task Monitor> In this puzzle, you need to explore the patterns of reaction by conducting continuous experiments(The law is simple and can be described in one sentense). Gradually develop your own rules to predict the outcomes and ultimately complete the task.You know from an incomplete list of reaction equations that: XY+Z = XZY. X, Y, and Z are placeholders representing different reactants or products, not specific substances.


Input anything to continue!
******************************************************
************Current puzzle progress: 2/15*************
Cur_puzzle_name: Reactor_puzzles, Cur_puzzle_level: 2, Cur_puzzle_index:12,Cur_buffer_size:0, Cur_Model_name: gpt-4o-2024-05-13, Do_abduction:True, Forced_abduction:None
******************************************************
------------------------------------------------Prompt------------------------------------------------
You are Kevin. You need to research and generate the corresponding chemical material required in the <Task Monitor> In this puzzle, you need to explore the patterns of reaction by conducting continuous experiments(The law is simple and can be described in one sentense). Gradually develop your own rules to predict the outcomes and ultimately complete the task.You know from an incomplete list of reaction equations that: XY+Z = XZY. X, Y, and Z are placeholders representing different reactants or products, not specific substances.


KeyboardInterrupt: Interrupted by user